Task 1: Data Understanding and Initial Insights


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import re
import os
from collections import Counter
import talib
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

##### LOAD AND EXAMINE STOCK DATA

In [2]:
def load_stock_data():
    """Load stock data and filter to the overlapping period (2011-04-27 to 2020-06-11)."""
    stock_symbols = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
    stock_data = {}
    start_date = pd.Timestamp('2011-04-27', tz='UTC').tz_convert('UTC-04:00')
    end_date = pd.Timestamp('2013-04-27', tz='UTC').tz_convert('UTC-04:00')
  #  start_date = pd.to_datetime('2011-04-27')
  #  end_date = pd.to_datetime('2020-06-11')
    for symbol in stock_symbols:
        try:
            filename = os.path.join('..', 'data', 'yfinance_data', f'{symbol}_historical_data.csv')            
            df = pd.read_csv(filename)
            df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize('UTC').dt.tz_convert('UTC-04:00')
            df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
            df['Symbol'] = symbol
            stock_data[symbol] = df
            print(f'✓ Loaded {symbol}: {len(df)} records (2011-04-27 to 2020-06-11)')
        except FileNotFoundError:
            print(f'✗ Could not load {symbol}_historical_data.csv')

    return stock_data
def load_news_data():
    """Load news data, infer missing dates from URLs, and deduplicate headlines."""
    try:
        filename = os.path.join('..', 'data','raw', 'raw_analyst_ratings.csv')

        news_df = pd.read_csv(filename)
        # Convert dates to datetime, handle timezone
        if 'date' in news_df.columns:
            news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')
        # Deduplicate headlines
        original_len = len(news_df)
        news_df = news_df.drop_duplicates(subset='headline', keep='first')
        print(f'✓ Deduplicated headlines: {original_len} to {len(news_df)} records')
        # Extract dates from URLs where 'date' is missing
        def extract_date_from_url(url):
            match = re.search(r'(\d{4}/\d{2}/\d{2})', url)
            if match:
                return pd.to_datetime(match.group(1), format='%Y/%m/%d', errors='coerce')
            return None
        # Apply date extraction for missing dates
        news_df['inferred_date'] = news_df.apply(
            lambda row: extract_date_from_url(row['url']) if pd.isna(row['date']) else row['date'], axis=1
        )
        # Use inferred dates where original dates are missing
        news_df['date'] = news_df['inferred_date'].combine_first(news_df['date'])
        news_df.drop(columns=['inferred_date'], inplace=True)
        # Filter to records with valid dates
        news_df = news_df.dropna(subset=['date'])
        print(f'Loaded news data: {len(news_df)} records after date imputation')
        return news_df
    except FileNotFoundError:
        print('Could not load raw_analyst_ratings.csv')
        return pd.DataFrame()

# Load the data
print('\n1. LOADING DATA')
print('-' * 30)
stock_data = load_stock_data()
news_data = load_news_data()


1. LOADING DATA
------------------------------
✓ Loaded AAPL: 503 records (2011-04-27 to 2020-06-11)
✓ Loaded AMZN: 503 records (2011-04-27 to 2020-06-11)
✓ Loaded GOOG: 503 records (2011-04-27 to 2020-06-11)
✓ Loaded META: 93 records (2011-04-27 to 2020-06-11)
✓ Loaded MSFT: 503 records (2011-04-27 to 2020-06-11)
✓ Loaded NVDA: 503 records (2011-04-27 to 2020-06-11)
✓ Loaded TSLA: 503 records (2011-04-27 to 2020-06-11)
✓ Deduplicated headlines: 1407328 to 845770 records
Loaded news data: 26939 records after date imputation


### BASIC DATA EXPLORATION

In [3]:
print('\nBASIC DATA STRUCTURE ANALYSIS')
# Examine stock data structure
if stock_data:
    sample_symbol = list(stock_data.keys())[0]
    sample_df = stock_data[sample_symbol]

    print(f'\nStock Data Structure (using {sample_symbol} as example):')
    print(f'Shape: {sample_df.shape}')
    print(f'Columns: {list(sample_df.columns)}')
    print(f'Date range: {sample_df["Date"].min()} to {sample_df["Date"].max()}')
    print(f'Data types:\n{sample_df.dtypes}')
    # Check for missing values
    print(f'\nMissing values:')
    print(sample_df.isnull().sum())

# Examine news data structure
if not news_data.empty:
    print(f'\n\nNews Data Structure:')
    print(f'Shape: {news_data.shape}')
    print(f'Columns: {list(news_data.columns)}')
    print(f'Date range: {news_data["date"].min()} to {news_data["date"].max()}')
    print(f'Data types:\n{news_data.dtypes}')

    # Check for missing values
    print(f'\nMissing values:')
    print(news_data.isnull().sum())


BASIC DATA STRUCTURE ANALYSIS

Stock Data Structure (using AAPL as example):
Shape: (503, 10)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']
Date range: 2011-04-26 20:00:00-04:00 to 2013-04-25 20:00:00-04:00
Data types:
Date            datetime64[ns, UTC-04:00]
Open                              float64
High                              float64
Low                               float64
Close                             float64
Adj Close                         float64
Volume                              int64
Dividends                         float64
Stock Splits                      float64
Symbol                             object
dtype: object

Missing values:
Date            0
Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
Dividends       0
Stock Splits    0
Symbol          0
dtype: int64


News Data Structure:
Shape: (26939, 6)
Columns: ['Unnamed: 0', 'head

### DESCRIPTIVE STATISTICS FOR STOCK DATA

In [4]:
def analyze_stock_statistics(stock_data):
    """Generate descriptive statistics for stock data within the aligned period."""
    all_stocks = pd.concat(stock_data.values(), ignore_index=True)

    print('\nOverall Stock Market Statistics (2011-2020):\n')
    numerical_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    available_cols = [col for col in numerical_cols if col in all_stocks.columns]
    desc_stats = all_stocks[available_cols].describe()
    print(desc_stats)

    print(f'\n--Analysis by Stock Symbol--\n')
    for symbol, df in stock_data.items():
        if 'Close' in df.columns:
            avg_price = df['Close'].mean()
            price_volatility = df['Close'].std()
            avg_volume = df['Volume'].mean()

            print(f'{symbol}:')
            print(f'  Average Close Price: ${avg_price:.2f}')
            print(f'  Price Volatility (σ): ${price_volatility:.2f}')
            print(f'  Average Volume: {avg_volume:,.0f}')
            print(f'  Date Range: {df["Date"].min().strftime("%Y-%m-%d")} to {df["Date"].max().strftime("%Y-%m-%d")}')
            print()

if stock_data:
    analyze_stock_statistics(stock_data)


Overall Stock Market Statistics (2011-2020):

              Open         High          Low        Close    Adj Close  \
count  3111.000000  3111.000000  3111.000000  3111.000000  3111.000000   
mean     12.946768    13.080579    12.804749    12.945495    11.587282   
std       9.913181    10.002246     9.819416     9.912698     8.377312   
min       0.287250     0.290750     0.278750     0.284500     0.260951   
25%       2.067000     2.115667     2.030000     2.077334     2.077334   
50%      13.075988    13.230907    12.901500    13.072003    12.311000   
75%      20.129725    20.308130    19.943679    20.137021    18.227549   
max      32.910000    32.950001    32.580002    32.849998    32.403412   

             Volume  
count  3.111000e+03  
mean   2.224539e+08  
std    2.702129e+08  
min    3.594000e+06  
25%    4.593070e+07  
50%    9.022000e+07  
75%    3.658360e+08  
max    3.195784e+09  

--Analysis by Stock Symbol--

AAPL:
  Average Close Price: $17.39
  Price Volatility (σ

### TIME SERIES ANALYSIS

In [ ]:
import plotly.io as pio
pio.renderers.default = 'browser'

In [5]:
def plot_time_series(stock_data, news_data):
    """Plot stock prices, volume, and news publication frequency for 2011-2020."""
    # Prepare news publication frequency
    if not news_data.empty:
        news_data['date_only'] = news_data['date'].dt.date
        daily_news_counts = news_data.groupby('date_only').size().reset_index(name='News Count')
        daily_news_counts['date_only'] = pd.to_datetime(daily_news_counts['date_only'])
    else:
        daily_news_counts = pd.DataFrame()

    # Create subplots
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        subplot_titles=('Close Price (2011-2020)', 'Volume (2011-2020)', 'News Publication Frequency'),
                        vertical_spacing=0.05)

    # Plot closing prices
    for symbol in stock_data:
        df = stock_data[symbol]
        fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], name=f'{symbol} Close', mode='lines'), row=1, col=1)

    # Plot volume
    for symbol in stock_data:
        df = stock_data[symbol]
        fig.add_trace(go.Scatter(x=df['Date'], y=df['Volume'], name=f'{symbol} Volume', mode='lines'), row=2, col=1)

    # Plot news publication frequency
    if not daily_news_counts.empty:
        fig.add_trace(go.Scatter(x=daily_news_counts['date_only'], y=daily_news_counts['News Count'],
                                 name='News Articles', mode='lines', line=dict(color='purple')), row=3, col=1)

    # Update layout
    fig.update_layout(height=1000, width=1000, title_text='Stock Price, Volume, and News Frequency (2011-2020)',
                      showlegend=True)
    fig.update_xaxes(title_text='Date', row=3, col=1)
    fig.update_yaxes(title_text='Close Price ($)', row=1, col=1)
    fig.update_yaxes(title_text='Volume (Shares)', row=2, col=1)
    fig.update_yaxes(title_text='News Articles', row=3, col=1)
    fig.show()

if stock_data and not news_data.empty:
    plot_time_series(stock_data, news_data)

### NEWS DATA ANALYSIS

In [6]:
def analyze_news_data(news_data):
    """Analyze news data with a focus on target stocks, headline keywords, and temporal trends."""
    if news_data.empty:
        print('No news data available for analysis')
        return

    print(f'Total news articles/ratings: {len(news_data)}')

    # Headline length analysis
    print(f'\nHeadline Analysis:')
    print(f'  Average headline length: {news_data["headline"].str.len().mean():.1f} characters')
    print(f'  Shortest headline: {news_data["headline"].str.len().min()} characters')
    print(f'  Longest headline: {news_data["headline"].str.len().max()} characters')

    # Keyword extraction from headlines
    print(f'\nKeyword Analysis:')
    words = ' '.join(news_data['headline']).lower().split()
    # Remove common stop words
    stop_words = {'the', 'a', 'an', 'and', 'or', 'to', 'in', 'on', 'for', 'with', 'at', 'by', 'from'}
    words = [word for word in words if word.isalnum() and word not in stop_words]
    common_words = Counter(words).most_common(10)
    print('Top 10 Keywords in Headlines:')
    for word, count in common_words:
        print(f'  {word}: {count} occurrences')

    # Publisher analysis
    print(f'\nPublisher Analysis:')
    publisher_counts = news_data['publisher'].value_counts().head(10)
    print('Top 10 Publishers:')
    for publisher, count in publisher_counts.items():
        print(f'  {publisher}: {count} articles')

    # Stock coverage for target stocks
    target_stocks = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
    print(f'\nStock Coverage Analysis (Target Stocks):')
    stock_counts = news_data[news_data['stock'].isin(target_stocks)]['stock'].value_counts()
    print('Articles per Target Stock Symbol:')
    for stock, count in stock_counts.items():
        print(f'  {stock}: {count} articles')

    # Temporal analysis
    print(f'\nTemporal Analysis:')
    print(f'  Date range: {news_data["date"].min()} to {news_data["date"].max()}')
    news_data['date_only'] = news_data['date'].dt.date
    daily_counts = news_data.groupby('date_only').size()
    print(f'  Average articles per day: {daily_counts.mean():.1f}')
    print(f'  Max articles in a day: {daily_counts.max()}')
    print(f'  Days with most activity: {daily_counts.nlargest(3).index.tolist()}')

if not news_data.empty:
    analyze_news_data(news_data)

Total news articles/ratings: 26939

Headline Analysis:
  Average headline length: 82.3 characters
  Shortest headline: 15 characters
  Longest headline: 512 characters

Keyword Analysis:
Top 10 Keywords in Headlines:
  of: 5753 occurrences
  eps: 5394 occurrences
  price: 4663 occurrences
  target: 4458 occurrences
  sales: 4194 occurrences
  maintains: 3250 occurrences
  shares: 3114 occurrences
  reports: 2860 occurrences
  q1: 2828 occurrences
  raises: 2206 occurrences

Publisher Analysis:
Top 10 Publishers:
  Benzinga Newsdesk: 10136 articles
  Paul Quintaro: 3802 articles
  Vick Meyer: 2126 articles
  ETF Professor: 1603 articles
  Charles Gross: 1444 articles
  Hal Lindon: 1291 articles
  Benzinga_Newsdesk: 1219 articles
  Eddie Staley: 918 articles
  vishwanath@benzinga.com: 918 articles
  Benzinga Insights: 498 articles

Stock Coverage Analysis (Target Stocks):
Articles per Target Stock Symbol:
  AAPL: 9 articles
  TSLA: 9 articles
  AMZN: 7 articles
  GOOG: 7 articles

Tempor

### DATA QUALITY ASSESSMENT

In [7]:
def assess_data_quality(stock_data, news_data):
    """Assess data quality for stock and news datasets."""
    print('Stock Data Quality:')
    for symbol, df in stock_data.items():
        missing_pct = (df.isnull().sum() / len(df)) * 100
        duplicate_dates = df['Date'].duplicated().sum()
        logical_errors = 0
        if all(col in df.columns for col in ['High', 'Low', 'Open', 'Close']):
            logical_errors += (df['High'] < df['Low']).sum()
            logical_errors += (df['High'] < df['Open']).sum()
            logical_errors += (df['High'] < df['Close']).sum()
            logical_errors += (df['Low'] > df['Open']).sum()
            logical_errors += (df['Low'] > df['Close']).sum()

        print(f'\n{symbol}:')
        print(f'  Missing data: {missing_pct.max():.1f}% (worst column)')
        print(f'  Duplicate dates: {duplicate_dates}')
        print(f'  Logical errors: {logical_errors}')
        completeness = 100 - missing_pct.max()
        consistency = 100 - (duplicate_dates/len(df)*100) - (logical_errors/len(df)*100)
        quality_score = (completeness + consistency) / 2
        print(f'  Quality Score: {quality_score:.1f}/100')

    if not news_data.empty:
        print(f'\n\nNews Data Quality:')
        print('-' * 18)
        missing_pct = (news_data.isnull().sum() / len(news_data)) * 100
        print(f'Missing data by column:')
        for col, pct in missing_pct.items():
            if pct > 0:
                print(f'  {col}: {pct:.1f}%')

        duplicate_headlines = news_data['headline'].duplicated().sum()
        print(f'Duplicate headlines: {duplicate_headlines}')

if stock_data:
    assess_data_quality(stock_data, news_data)

Stock Data Quality:

AAPL:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

AMZN:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

GOOG:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

META:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

MSFT:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

NVDA:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100

TSLA:
  Missing data: 0.0% (worst column)
  Duplicate dates: 0
  Logical errors: 0
  Quality Score: 100.0/100


News Data Quality:
------------------
Missing data by column:
Duplicate headlines: 0


### QUANTITATIVE ANALYSIS (TASK 2 PREPARATION)

In [8]:
def calculate_technical_indicators(stock_data):
    """Calculate technical indicators (MA, RSI, MACD) using TA-Lib."""
    for symbol, df in stock_data.items():
        df = df.copy()  # Avoid SettingWithCopyWarning
        df['MA50'] = talib.SMA(df['Close'], timeperiod=50)
        df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
        df['MACD'], df['MACD_signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        stock_data[symbol] = df
        print(f'✓ Calculated technical indicators for {symbol}')

    # Plot technical indicators for a sample stock (AAPL)
    sample_df = stock_data['AAPL']
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        subplot_titles=('Close Price with MA50', 'RSI', 'MACD'),
                        vertical_spacing=0.05)

    # Close Price and MA50
    fig.add_trace(go.Scatter(x=sample_df['Date'], y=sample_df['Close'], name='Close Price', mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=sample_df['Date'], y=sample_df['MA50'], name='MA50', mode='lines', line=dict(color='orange')), row=1, col=1)

    # RSI
    fig.add_trace(go.Scatter(x=sample_df['Date'], y=sample_df['RSI'], name='RSI', mode='lines', line=dict(color='green')), row=2, col=1)
    fig.add_shape(type='line', x0=sample_df['Date'].min(), x1=sample_df['Date'].max(), y0=70, y1=70,
                  line=dict(color='red', dash='dash'), row=2, col=1)
    fig.add_shape(type='line', x0=sample_df['Date'].min(), x1=sample_df['Date'].max(), y0=30, y1=30,
                  line=dict(color='red', dash='dash'), row=2, col=1)

    # MACD
    fig.add_trace(go.Scatter(x=sample_df['Date'], y=sample_df['MACD'], name='MACD', mode='lines', line=dict(color='blue')), row=3, col=1)
    fig.add_trace(go.Scatter(x=sample_df['Date'], y=sample_df['MACD_signal'], name='MACD Signal', mode='lines', line=dict(color='purple')), row=3, col=1)

    fig.update_layout(height=1000, width=1000, title_text='Technical Indicators for AAPL (2011-2020)', showlegend=True)
    fig.update_xaxes(title_text='Date', row=3, col=1)
    fig.update_yaxes(title_text='Price ($)', row=1, col=1)
    fig.update_yaxes(title_text='RSI', row=2, col=1)
    fig.update_yaxes(title_text='MACD', row=3, col=1)
    fig.show()

if stock_data:
    calculate_technical_indicators(stock_data)

✓ Calculated technical indicators for AAPL
✓ Calculated technical indicators for AMZN
✓ Calculated technical indicators for GOOG
✓ Calculated technical indicators for META
✓ Calculated technical indicators for MSFT
✓ Calculated technical indicators for NVDA
✓ Calculated technical indicators for TSLA


### SUMMARY AND INSIGHTS

### KEY INSIGHTS AND FINDINGS

1. **DATA AVAILABILITY**:
   - Stock data successfully filtered to 2011-04-27 to 2020-06-11, aligning with news data.
   - News data reduced to records with valid dates after imputation, focusing on actionable insights.

2. **DATA QUALITY**:
   - Stock data remains high quality with no missing values or logical errors.
   - News data quality improved by deduplicating headlines and inferring missing dates from URLs.

3. **MARKET INSIGHTS**:
   - Price trends show significant movements (e.g., AAPL growth over the decade, NVDA volatility tied to high trading volume).
   - News publication spikes (e.g., 2020-03-12) align with market events like the COVID-19 crash, suggesting potential correlation with price movements.
   - Keywords like 'earnings' and 'price target' dominate headlines, indicating focus areas for sentiment analysis.

4. **READINESS FOR ANALYSIS**:
   - Technical indicators (MA50, RSI, MACD) calculated, providing a foundation for Task 2.
   - Time series alignment enables correlation analysis between news sentiment and stock returns in Task 3.
   - Data is now better prepared for deeper quantitative and sentiment analysis.